In [157]:
import random
import math
import os
import subprocess
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [158]:

def generate_random_qiskit_circuit(num_qubits, num_operations, filename):
    code = f"""import math
from qiskit import *

q = QuantumRegister({num_qubits}, 'q')
c = ClassicalRegister({num_qubits}, 'c')
qc = QuantumCircuit(q, c)

"""
    
    gates = ["x", "h", "p", "t", "s", "z", "y", "id", "rx", "ry", "rz", "sx", "swap", "rzz", "rxx", "cx", "cz", "ccx", "cswap"]
    
    for _ in range(num_operations):
        gate = random.choice(gates)
        
        if gate in ['x', 'h', 't', 's', 'z', 'y', 'id', 'sx']:
            qubit = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{qubit}])\n"
        elif gate in ['p', 'rx', 'ry', 'rz']:
            qubit = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit}])\n"
        elif gate in ['cx', 'cz', 'swap']:
            control = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while target == control:
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target}])\n"
        elif gate in ['rzz', 'rxx']:
            qubit1 = random.randint(0, num_qubits - 1)
            qubit2 = random.randint(0, num_qubits - 1)
            while qubit2 == qubit1:
                qubit2 = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit1}], q[{qubit2}])\n"
        elif gate == 'ccx':
            control1 = random.randint(0, num_qubits - 1)
            control2 = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while control2 == control1 or target == control1 or target == control2:
                control2 = random.randint(0, num_qubits - 1)
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control1}], q[{control2}], q[{target}])\n"
        elif gate == 'cswap':
            control = random.randint(0, num_qubits - 1)
            target1 = random.randint(0, num_qubits - 1)
            target2 = random.randint(0, num_qubits - 1)
            while target1 == control or target2 == control or target1 == target2:
                target1 = random.randint(0, num_qubits - 1)
                target2 = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target1}], q[{target2}])\n"
    
    with open(filename, 'w') as f:
        f.write(code)

### Generate random circuits in "Testcases" directory

In [159]:
import os

# Define base and subdirectories using os.path.join for cross-platform compatibility
base_dir = "Testcases"
deep_dir = os.path.join(base_dir, "deep")
shallow_dir = os.path.join(base_dir, "shallow")

# Create directories if they do not already exist
os.makedirs(deep_dir, exist_ok=True)
os.makedirs(shallow_dir, exist_ok=True)

# Generate the random Qiskit circuits in the respective directories
for i in range(3):
    generate_random_qiskit_circuit(3, 3, os.path.join(deep_dir, f"deep_{i}.py"))    


In [160]:
import os
import subprocess

# Define the config file path.
config = os.path.join("muskit", "generatorConfig.py")

# Create the output directory for shallow mutants.
mutants_dir = os.path.join("Mutants", "deep")
os.makedirs(mutants_dir, exist_ok=True)

# Define the testcases directory.
testcases_dir = os.path.join("Testcases", "deep")

for file in os.listdir(testcases_dir):
    if file.endswith(".py"):
        name = os.path.splitext(file)[0]
        process = subprocess.Popen(
            [
                "python",
                "-m", "muskit.ComandMain",
                "Create",
                config,
                os.path.join(testcases_dir, file),
                os.path.join(mutants_dir, f"mutants_of_{name}")
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        process.wait()

In [161]:
import os
import subprocess
from tqdm import tqdm

# Build file paths using os.path.join for portability
config = os.path.join("muskit", "executorConfig.py")
inputs = os.path.join("muskit", "testcases.py")
mutants_dir = os.path.join("Mutants", "deep")

for file in tqdm(os.listdir(mutants_dir), desc="Processing all mutants for each shallow circuit"):
    try:
        # Remove the "mutants_of_" prefix to get the name
        name = file[len("mutants_of_"):]
        # Build the save path using os.path.join and an f-string for clarity
        savePath = os.path.join("Bias", "deep", f"{name}.txt")
        
        # Run the process with the properly constructed paths
        process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Execute",
            config,
            inputs,
            os.path.join(mutants_dir, file),
            savePath
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        print(stdout)
        if stderr:
            print("Errors:", stderr)
    except Exception as e:
        print(f"Error processing file {file}: {str(e)}")


Processing all mutants for each shallow circuit:  33%|███▎      | 1/3 [01:05<02:11, 65.85s/it]

Mutants/deep/mutants_of_deep_1
Mutants/deep/mutants_of_deep_1
Testcases/deep/deep_1.py
Processing mutant file: Mutants/deep/mutants_of_deep_1/19AddGate_cswap_inGap_1_.py
Original counts: {'010': 493, '000': 507}
Mutant counts: {'010': 483, '000': 517}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/15AddGate_rxx_inGap_1_.py
Original counts: {'000': 478, '010': 522}
Mutant counts: {'000': 514, '010': 486}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/17AddGate_cz_inGap_1_.py
Original counts: {'000': 515, '010': 485}
Mutant counts: {'000': 536, '010': 464}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/38AddGate_cswap_inGap_5_.py
Original counts: {'010': 485, '000': 515}
Mutant counts: {'010': 510, '000': 490}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/7AddGate_y_inGap_1_.py
Original counts: {'010': 498, '000': 502}
Mutant counts: {'011': 489, '001': 511}
Calculated bias: 1.0
P

Processing all mutants for each shallow circuit:  67%|██████▋   | 2/3 [02:13<01:06, 66.70s/it]

Mutants/deep/mutants_of_deep_0
Mutants/deep/mutants_of_deep_0
Testcases/deep/deep_0.py
Processing mutant file: Mutants/deep/mutants_of_deep_0/19AddGate_cswap_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/15AddGate_rxx_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 512, '110': 488}
Calculated bias: 0.488
Processing mutant file: Mutants/deep/mutants_of_deep_0/17AddGate_cz_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/38AddGate_cswap_inGap_5_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/7AddGate_y_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'010': 1000}
Calculated bias: 1.0
Processing mutant file: Mutants/deep/mutants_of_deep_0/24AddGate_s_inGap_5_.py
Original counts: 

Processing all mutants for each shallow circuit: 100%|██████████| 3/3 [03:23<00:00, 67.68s/it]

Mutants/deep/mutants_of_deep_2
Mutants/deep/mutants_of_deep_2
Testcases/deep/deep_2.py
Processing mutant file: Mutants/deep/mutants_of_deep_2/19AddGate_cswap_inGap_1_.py
Original counts: {'000': 769, '110': 231}
Mutant counts: {'110': 231, '000': 769}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/15AddGate_rxx_inGap_1_.py
Original counts: {'110': 247, '000': 753}
Mutant counts: {'000': 69, '110': 931}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/17AddGate_cz_inGap_1_.py
Original counts: {'110': 239, '000': 761}
Mutant counts: {'110': 270, '000': 730}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/38AddGate_cswap_inGap_5_.py
Original counts: {'110': 252, '000': 748}
Mutant counts: {'101': 260, '000': 740}
Calculated bias: 0.26
Processing mutant file: Mutants/deep/mutants_of_deep_2/7AddGate_y_inGap_1_.py
Original counts: {'110': 234, '000': 766}
Mutant counts: {'010': 222, '100': 778}
Calculated bias: 1.0